- Определяем тиер в зависимости от ранга команды в мировом рейтинге, мол тиер -1 первые 15-25 команды, к примеру и выводим результат
- Важность турнира пока не знаю как вывести, но можно собрать инфу по количеству выигрываемых денег а также по тому, какие команды играют
- Рейтинг игроков я думаю пока средний 2.0 выводить
- Активность команды выводим по количеству сыгранных игр 
- История встреч, сыгранные карты и счёт на них
- История игр с одинаковыми врагами (типа вп - фазе, ликвид фазе, когда сравниваем вп и ликвид)
- Мапы пока думаю просто выводить списком, которые сортируется, мол сыграли с тир-1 командой, тир-2 и так далее командами, результат, противник, карту (тоже сортируем) и метку поражения или победы

In [169]:
import requests
import re

# получаем страницу игры по ссылке и выбираем карту (с маленькой буквы)
maps = 'inferno'
link = 'https://www.hltv.org/matches/2337359/nip-vs-big-esl-pro-league-season-10-europe'
request = requests.get(link).text

# функция по определению ранга команды
pattern_findrank = r"<a href=\"/ranking/teams\">#([\w-]+)<"
def get_rank(team_page):
    return re.findall(pattern_findrank, team_page)


# функция по определению тира команды
def get_tier(team_page):
    rank = get_rank(team_page)
    if len(rank) == 0:
        return 3
    else:
        if int(rank[0]) <= 15:
            return 1
        elif int(rank[0]) <=30:
            return 2
        else:
            return 3
    
    
# получаем название и страницу турнира
pattern_find_tournament = r"<div class=\"event text-ellipsis\"><a href=\"(.*)\" title=\"(.*)\""
tournament = re.findall(pattern_find_tournament, request)
tournament_link = 'https://www.hltv.org' + tournament[0][0]
tournament_name = tournament[0][1]
tournament_page = requests.get(tournament_link).text
print('Tournament:', tournament_name)


# получаем информацию о награде турнира
pattern_prize = r"<td class=\"prizepool text-ellipsis\" title=\"([\w\s$,+\-]+)\""
prize = re.findall(pattern_prize, tournament_page)[0]
print('Prize: ', prize)


# получаем тир турнира
pattern_tournament_teams = r"<a href=\"/team([\w/-]+)\" data-link-tracking-page=\"Eventpage\""
tournament_teams = re.findall(pattern_tournament_teams, tournament_page)
sum_rank = 0
for i in tournament_teams:
    page_team = requests.get('https://www.hltv.org/team' + i).text
    sum_rank += get_tier(page_team)
tournament_tier = sum_rank / len(tournament_teams)
print('Teams:', len(tournament_teams))
print('Tournament Tier:', round(tournament_tier, 1))


# получаем информацию о стадии турнира
pattern_tournament_stage = r"<div class=\"padding preformatted-text\">[\w\s]+\* ([\w\s]+).*<"
tournament_stage = re.findall(pattern_tournament_stage, request)
if len(tournament_stage) > 0:
    print('Tournament Stage:', tournament_stage[0])

    
# получаем историю пиков и банов для начавшейся игры
pattern_history_pick = r"<div>\d\. (.*)</div>"
history_pick = re.findall(pattern_history_pick, request)
if len(history_pick) > 0:
    print('----------------------------------------------------------')
    print('History picks and bans:')
    for i in history_pick:
        print('-', i)

        
# получаем название команд 
pattern_findteam = r"class=\"teamName\">([\w\s\.-]+)<"
teams = re.findall(pattern_findteam, request)
team1 = teams[0]
team2 = teams[1]


# получаем страницы команд
pattern_team1 = r"<div class=\"team1-gradient\"><a href=\"([\w/-]+)\""
team1_link = 'https://www.hltv.org' + re.findall(pattern_team1, request)[0] 
request1 = requests.get(team1_link).text
pattern_team2 = r"<div class=\"team2-gradient\"><a href=\"([\w/-]+)\""
team2_link = 'https://www.hltv.org' + re.findall(pattern_team2, request)[0]
request2 = requests.get(team2_link).text


# получаем тир и ранг команд
rank1 = get_rank(request1)
rank2 = get_rank(request2)
tier1 = get_tier(request1)
tier2 = get_tier(request2)


# получаем информацию о командах с аналитики hltv
pattern_analytisc_hltv = r"<a href=\"(.*)\" class=\"matchpage-analytics-center-container\">"
analytisc_hltv_link = 'https://www.hltv.org' + re.findall(pattern_analytisc_hltv, request)[0]
analytisc_hltv = requests.get(analytisc_hltv_link).text
pattern_info_team1 = r"<div class=\"analytics-insights-container team1\">([\s\S]*?)<div class=\"col-6\">"
info_team1 = re.findall(pattern_info_team1, analytisc_hltv)[0]
pattern_info_team2 = r"<div class=\"analytics-insights-container team2\">([\s\S]*?)<div class=\"analytics-bookmakers g-grid\" id=\"bookmakers\">"
info_team2 = re.findall(pattern_info_team2, analytisc_hltv)[0]
pattern_find_plus = r"<div class=\"analytics-insights-indicator in-favor\"><i class=\"fa fa-plus*\"></i></div>\n\s*<div class=\"analytics-insights-info\">(.*)</div>"
find_plus1 = re.findall(pattern_find_plus, info_team1)
find_plus2 = re.findall(pattern_find_plus, info_team2)
pattern_find_minus = r"<div class=\"analytics-insights-indicator against\"><i class=\"fa fa-minus\"></i></div>\n\s*<div class=\"analytics-insights-info\">(.*)</div>"
find_minus1 = re.findall(pattern_find_minus, info_team1)
find_minus2 = re.findall(pattern_find_minus, info_team2)
print('----------------------------------------------------------')
print('Team information')
print(team1)
print('Tier:', tier1)
print('Rank:', rank1[0])
if len(find_plus1) != 0:
    for i in find_plus1:
        print('+', i)
else:
    print('+ No insights detected for', team1)
if len(find_minus1) != 0:
    for i in find_minus1:
        print('-', i)
else:
    print('- No insights detected for', team1)
print()
print(team2)
print('Tier:', tier2)
print('Rank:', rank2[0])
if len(find_plus2) != 0:
    for i in find_plus2:
        print('+', i)
else:
    print('+ No insights detected for', team2)
if len(find_minus2) != 0:
    for i in find_minus2:
        print('-', i)
else:
    print('- No insights detected for', team2)


# получаем форму игроков с аналитики hltv
pattern_players_form1 =  r"<div class=\"analytics-head-to-head-container team1\">([\s\S]*?)<div class=\"analytics-last-matches\">"    
pattern_players_form2 =  r"<div class=\"analytics-head-to-head-container team2\">([\s\S]*?)<div class=\"analytics-last-matches\">"    
players_form1 = re.findall(pattern_players_form1, analytisc_hltv)[0]
players_form2 = re.findall(pattern_players_form2, analytisc_hltv)[0]
pattern_find_player = r"<div class=\"player-nickname\">(.*)</div>"
pattern_form_latest = r"<td class=\"table-3-months[\s\w]*?\">(.*)</td>"
pattern_form_event = r"<td class=\"table-event[\s\w]*?\"[\s\S]*?>(.*)</td>"
find_player1 = re.findall(pattern_find_player, players_form1)
form_latest1 = re.findall(pattern_form_latest, players_form1)
form_event1 = re.findall(pattern_form_event, players_form1)
find_player2 = re.findall(pattern_find_player, players_form2)
form_latest2 = re.findall(pattern_form_latest, players_form2)
form_event2 = re.findall(pattern_form_event, players_form2)
print('----------------------------------------------------------')
print('Form players (Nick | 3 months | Event)')
print(team1)
i = 0
while i < len(find_player1):
    print(find_player1[i], '|', form_latest1[i], '|', form_event1[i])
    i += 1
print()
print(team2)
i = 0
while i < len(find_player2):
    print(find_player2[i], '|', form_latest2[i], '|', form_event2[i])
    i += 1

    
# получаем ссылки на сыгранные карты за последние 3 месяца
pattern_stats_team = r"<a href=\"(/stats/teams/map/.*)\" class=\"moreButton\""
link_maps1 = re.findall(pattern_stats_team, request1)
link_maps2 = re.findall(pattern_stats_team, request2)


# получаем страницу с искомой картой
card_code = {'dust2':'/31/', 'mirage':'/32/', 'inferno':'/33/', 'nuke':'/34/', 'train':'/35/', 'overpass':'/40/', 'vertigo':'/46/'}
pattern_code = r"(/\d\d/)"
score_link = 'https://www.hltv.org' + link_maps1[0]
score_link1 = re.sub(pattern_code, card_code[maps], score_link)
score_link1 = score_link1.replace("amp;", "")
score_page1 = requests.get(score_link1).text
score_link = 'https://www.hltv.org' + link_maps2[0]
score_link2 = re.sub(pattern_code, card_code[maps], score_link)
score_link2 = score_link2.replace("amp;", "")
score_page2 = requests.get(score_link2).text


# получаем статистику по сыгранной карте
pattern_times_played = r"<div class=\"stats-row\"><span class=\"strong\">Times played</span><span>(\w*)<"
times_played1 = re.findall(pattern_times_played, score_page1)[0]
times_played2 = re.findall(pattern_times_played, score_page2)[0]
pattern_wins_draws_losses = r"<div class=\"stats-row\"><span class=\"strong\">Wins / draws / losses</span><span>(\w*) / (\w*) / (\w*)<"
wins_draws_losses1 = re.findall(pattern_wins_draws_losses, score_page1)
wins_draws_losses2 = re.findall(pattern_wins_draws_losses, score_page2)
pattern_total_rounds_played = r"<div class=\"stats-row\"><span class=\"strong\">Total rounds played</span><span>(\w*)<"
total_rounds_played1 = re.findall(pattern_total_rounds_played, score_page1)[0]
total_rounds_played2 = re.findall(pattern_total_rounds_played, score_page2)[0]
patter_rounds_won = r"<div class=\"stats-row\"><span class=\"strong\">Rounds won</span><span>(\w+)<"
rounds_won1 = re.findall(patter_rounds_won, score_page1)[0]
rounds_won2 = re.findall(patter_rounds_won, score_page2)[0]
pattern_win_percent = r"<div class=\"stats-row\"><span class=\"strong\">Win percent</span><span class=\"\w*\">([\w\.%]*)<"
win_percent1 = re.findall(pattern_win_percent, score_page1)[0]
win_percent2 = re.findall(pattern_win_percent, score_page2)[0]
win_round_percent1 = str(round((int(rounds_won1) / int(total_rounds_played1) * 100), 1)) + '%'
win_round_percent2 = str(round((int(rounds_won2) / int(total_rounds_played2) * 100), 1)) + '%'
pattern_distribution_of_won_rounds = r"data&quot;:\[\{&quot;label&quot\;\:&quot;\w*&quot;,&quot;value&quot;:&quot;(\w*)&quot;,&quot;color&quot;:&quot;2C6EA4&quot;\},\{&quot;label&quot;:&quot;\w*&quot;,&quot;value&quot;:&quot;(\w*)&quot;"
distribution_of_won_rounds1 = re.findall(pattern_distribution_of_won_rounds, score_page1)
distribution_of_won_rounds2 = re.findall(pattern_distribution_of_won_rounds, score_page2)
win_ct_1 = str(round((int(distribution_of_won_rounds1[0][0]) / (int(distribution_of_won_rounds1[0][0]) + int(distribution_of_won_rounds1[0][1])) * 100), 2)) + '%' 
win_t_1 = str(round((int(distribution_of_won_rounds1[0][1]) / (int(distribution_of_won_rounds1[0][0]) + int(distribution_of_won_rounds1[0][1])) * 100), 2)) + '%' 
win_ct_2 = str(round((int(distribution_of_won_rounds2[0][0]) / (int(distribution_of_won_rounds2[0][0]) + int(distribution_of_won_rounds2[0][1])) * 100), 2)) + '%' 
win_t_2 = str(round((int(distribution_of_won_rounds2[0][1]) / (int(distribution_of_won_rounds2[0][0]) + int(distribution_of_won_rounds2[0][1])) * 100), 2)) + '%' 
pattern_pistol_rounds = r"<div class=\"stats-row\"><span class=\"strong\">Pistol rounds</span><span>(\w*)<"
pistol_rounds1 = re.findall(pattern_pistol_rounds, score_page1)[0]
pistol_rounds2 = re.findall(pattern_pistol_rounds, score_page2)[0]
pattern_pistol_rounds_won = r"<div class=\"stats-row\"><span class=\"strong\">Pistol rounds won</span><span>(\w*)<"
pistol_rounds_won1 = re.findall(pattern_pistol_rounds_won, score_page1)[0]
pistol_rounds_won2 = re.findall(pattern_pistol_rounds_won, score_page2)[0]
pattern_pistol_round_win_percent = r"<div class=\"stats-row\"><span class=\"strong\">Pistol round win percent</span><span class=\"\w*\">([\w\.%]*)<"
pistol_round_win_percent1 = re.findall(pattern_pistol_round_win_percent, score_page1)[0]
pistol_round_win_percent2 = re.findall(pattern_pistol_round_win_percent, score_page2)[0]


# получаем статистику сыгранных матчей на выбранной карте
pattern_match_date = r"<td class=\"time\"><a href=\".*\">([\d/]*)<"
match_date1 = re.findall(pattern_match_date, score_page1)
match_date2 = re.findall(pattern_match_date, score_page2)
pattern_score_match = r"<td class=\"statsTeamMapResult match-(\w*)\">(.*)</td>"
score_match1 = re.findall(pattern_score_match, score_page1)
score_match2 = re.findall(pattern_score_match, score_page2)
pattern_opponent = r"<td><a href=\".*\" class=\"image-and-label\".*><span>(.*)</span></a>"
opponent1 = re.findall(pattern_opponent, score_page1)
opponent2 = re.findall(pattern_opponent, score_page2)
pattern_opponent_link = r"<td><a href=\"/stats/teams/(\d*)/.*?\" class=\"image-and-label\" data-tooltip-id=\".*?\">"
opponent_link1 = re.findall(pattern_opponent_link, score_page1)
opponent_link2 = re.findall(pattern_opponent_link, score_page2)
ranks_opponent1 = []
ranks_opponent2 = []
i = 0
while i < len(opponent1):
    opponent = opponent1[i].lower()
    opponent = opponent.replace(' ', '-')
    page_opponent = requests.get('https://www.hltv.org/team/' + opponent_link1[i] + '/opponent').text
    rank = get_rank(page_opponent)
    if len(rank) == 0:
        ranks_opponent1.append('-')
    else:
        ranks_opponent1.append(rank[0])
    i += 1
i = 0
while i < len(opponent2):
    opponent = opponent2[i].lower()
    opponent = opponent.replace(' ', '-')
    page_opponent = requests.get('https://www.hltv.org/team/' + opponent_link2[i] + '/opponent').text
    rank = get_rank(page_opponent)
    if len(rank) == 0:
        ranks_opponent2.append('-')
    else:
        ranks_opponent2.append(rank[0])
    i += 1
pattern_match_tournament = r"<td class=\"\"><a href=\".*\" class=\"image-and-label\" data-tooltip-id=\".*\" title=\"(.*)\">"
match_tournament1 = re.findall(pattern_match_tournament, score_page1)
match_tournament2 = re.findall(pattern_match_tournament, score_page2)

print('----------------------------------------------------------')
print(team1)
print(maps, 'stats')
print('Times played:', times_played1)
print('Wins/draws/losses:', wins_draws_losses1[0][0], wins_draws_losses1[0][1], wins_draws_losses1[0][2])
print('Total rounds played:', total_rounds_played1)
print('Rounds won:', rounds_won1)
print('Win percent:', win_percent1)
print('Win round percent:', win_round_percent1)
print('Win CT percent:', win_ct_1)
print('Win T percent:', win_t_1)
print('Pistol rounds:', pistol_rounds1)
print('Pistol rounds won:', pistol_rounds_won1)
print('Pistol round win percent:', pistol_round_win_percent1)

print()
print(team2)
print(maps, 'stats')
print('Times played:', times_played2)
print('Wins/draws/losses:', wins_draws_losses2[0][0], wins_draws_losses2[0][1], wins_draws_losses2[0][2])
print('Total rounds played:', total_rounds_played2)
print('Rounds won:', rounds_won2)
print('Win percent:', win_percent2)
print('Win round percent:', win_round_percent2)
print('Win CT percent:', win_ct_2)
print('Win T percent:', win_t_2)
print('Pistol rounds:', pistol_rounds2)
print('Pistol rounds won:', pistol_rounds_won2)
print('Pistol round win percent:', pistol_round_win_percent2)

print('----------------------------------------------------------')
print('Mathes')
print(team1)
i = 0
while i < len(match_date1):
    print(match_date1[i], '|', score_match1[i][0], '|', score_match1[i][1], '| Rank:', ranks_opponent1[i], opponent1[i], '|', match_tournament1[i])
    i += 1
i = 0
print()
print(team2)
while i < len(match_date2):
    print(match_date2[i], '|', score_match2[i][0], '|', score_match2[i][1], '| Rank:', ranks_opponent2[i], opponent2[i], '|', match_tournament2[i])
    i += 1

Tournament: ESL Pro League Season 10 Europe
Prize:  TBA
Teams: 16
Tournament Tier: 1.7
Tournament Stage: Group A
----------------------------------------------------------
History picks and bans:
- BIG removed Vertigo
- NiP removed Dust2
- BIG picked Train
- NiP picked Nuke
- NiP removed Overpass
- BIG removed Mirage
- Inferno was left over
----------------------------------------------------------
Team information
NiP
Tier: 1
Rank: 8
+ NiP played 1 matches against BIG in the past 30 days, they won 1 and lost 0
+ NiP has better form ranking
+ NiP is the bookmaker favorite
+ NiP is better ranked (#8)
- No insights detected for NiP

BIG
Tier: 2
Rank: 24
+ BIG has won 4 out of the last 5 matches
- BIG is worse ranked (#24)
----------------------------------------------------------
Form players (Nick | 3 months | Event)
NiP
Plopski | 1.14 | 1.19
f0rest | 1.09 | 1.18
REZ | 1.02 | 1.13
twist | 1.09 | 1.05
Lekr0 | 1.06 | 0.96

BIG
tabseN | 1.20 | 1.09
XANTARES | 1.18 | 1.09
smooya | 1.06 | 1.